In [29]:
# Use with NHL_Model_Data_Transform_v5.py
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras


rng = 69

In [30]:
#df = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Over_Under_Transformed.csv")
df = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Over_Under_Transformed.csv")

In [31]:
df = df.drop(columns=['team_Home','team_Away','gameId','home_or_away_Home','home_or_away_Away','win_or_lose_Away','win_or_lose_Home','goalsTotal_Away'])

In [32]:
# df.hist(bins=50, figsize=(20,15))
# plt.show()

In [33]:
# Heatmap to visualize correlations
corr_matrix = df.corr(numeric_only=True)
# sns.heatmap(corr_matrix, cmap="PiYG")

In [34]:
corr_matrix['goalsTotal_Home'].sort_values(ascending=False)

goalsTotal_Home              1.000000
goalsAgainstAvg_Home         0.052154
goalsForAvg_Away             0.048619
goalsAgainstAvg_Away         0.040320
reboundsForAvg_Away          0.020040
reboundsForAvg_Home          0.016664
penaltiesForTotal            0.011857
fenwickPercentageAvg_Away    0.003288
penaltiesAgainstTotal        0.001298
pointsFromGameAvg_Away      -0.002941
goalsForAvg_Home            -0.003039
seasonPointsPerGame_Away    -0.006146
hitsDiffAvg_Away            -0.007814
fenwickPercentageAvg_Home   -0.008797
seasonPointsPerGame_Home    -0.022595
pointsFromGameAvg_Home      -0.024717
hitsDiffAvg_Home            -0.029000
Name: goalsTotal_Home, dtype: float64

In [35]:
# Visualize distributions of attributes
# from pandas.plotting import scatter_matrix

# attributes = ['fenwickPercentageAvg_Home','goalDiffAvg_Home','seasonPointsPerGame_Home','seasonPointsPerGame_Away']
# scatter_matrix(df[attributes], figsize=(12,8));

In [36]:
# These are tentative just to test
df_Final = df
# df_Final = df.drop(columns=['corsiPercentageAvg_Away','penaltiesAgainstTotal','shotsOnGoalDiffAvg_Away'])

In [37]:
from sklearn.model_selection import train_test_split

# Split into 80% training data and 20% testing data
train_set, test_set = train_test_split(df_Final, test_size=0.2, random_state=rng)

nhl = train_set.drop('goalsTotal_Home', axis=1)
nhl_labels = train_set['goalsTotal_Home'].copy()

X_train = train_set.drop('goalsTotal_Home', axis=1)
Y_train = train_set['goalsTotal_Home'].copy()

X_valid = train_set.drop('goalsTotal_Home', axis=1)
Y_valid = train_set['goalsTotal_Home'].copy()

X_test = test_set.drop('goalsTotal_Home', axis=1)
Y_test = test_set['goalsTotal_Home'].copy()

df_Final = df_Final.drop('goalsTotal_Home', axis=1)

In [38]:
# Pipeline constructor used to run transformation steps in order
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])

X_train_prepared= num_pipeline.fit_transform(X_train)
X_valid_prepared= num_pipeline.fit_transform(X_valid)
X_test_prepared= num_pipeline.fit_transform(X_test)


In [39]:
n_epoch = 150

In [40]:
def build_model(n_neurons=50, n_hidden=3, dropout_rate=0.4, learning_rate=1e-3, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='selu'))
        model.add(keras.layers.Dropout(dropout_rate))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['mae'])
    return model

In [41]:
from scikeras.wrappers import KerasRegressor

# Seems like you need to specify some items again when using the scikeras wrapper
keras_reg = KerasRegressor(model=build_model, n_hidden=1, n_neurons=30, learning_rate=3e-3)


In [42]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    'model__n_hidden': [0,1,2,3,4,5],
    'model__dropout_rate': [0,.1,.2,.3,.4],
    'model__n_neurons': np.arange(10,60),
    'model__learning_rate' : reciprocal(1e-5, 1e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=20, cv=3, scoring='neg_mean_squared_error')

early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True)
rnd_search_cv.fit(X_train_prepared,Y_train, epochs=n_epoch,
                 validation_data = (X_valid_prepared, Y_valid),
                 callbacks=[early_stopping_cb])

Epoch 1/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 41.6620 - mae: 5.9545 - val_loss: 38.6892 - val_mae: 5.7078
Epoch 2/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37.4035 - mae: 5.6041 - val_loss: 34.3433 - val_mae: 5.3401
Epoch 3/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 32.9232 - mae: 5.2299 - val_loss: 29.7093 - val_mae: 4.9068
Epoch 4/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.8961 - mae: 4.7128 - val_loss: 24.8634 - val_mae: 4.4083
Epoch 5/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23.2910 - mae: 4.2413 - val_loss: 19.9061 - val_mae: 3.8456
Epoch 6/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18.0477 - mae: 3.6049 - val_loss: 15.3702 - val_mae: 3.2758
Epoch 7/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.4719 - mae: 3.0232 - val_loss: 11.6197 - val_mae: 2.7656
Epoch 8/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10.8638 - mae: 2.6349 - val_loss: 8.8845 - val_mae: 2.3639
Epoch 9/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/ste

Epoch 55/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.3158 - mae: 1.8514 - val_loss: 5.3936 - val_mae: 1.8502
Epoch 56/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.2084 - mae: 1.8020 - val_loss: 5.3845 - val_mae: 1.8477
Epoch 57/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.2926 - mae: 1.8026 - val_loss: 5.3835 - val_mae: 1.8469
Epoch 58/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.1287 - mae: 1.8089 - val_loss: 5.3872 - val_mae: 1.8471
Epoch 59/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.2836 - mae: 1.8364 - val_loss: 5.3826 - val_mae: 1.8464
Epoch 60/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.4190 - mae: 1.8689 - val_loss: 5.3775 - val_mae: 1.8462
Epoch 61/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.3204 - mae: 1.8460 - val_loss: 5.3793 - val_mae: 1.8462
Epoch 62/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.1237 - mae: 1.7946 - val_loss: 5.3801 - val_mae: 1.8476
Epoch 63/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - lo

Epoch 109/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.1995 - mae: 1.8251 - val_loss: 5.3079 - val_mae: 1.8340
Epoch 110/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.1157 - mae: 1.8169 - val_loss: 5.3113 - val_mae: 1.8344
Epoch 111/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.1894 - mae: 1.8074 - val_loss: 5.3079 - val_mae: 1.8353
Epoch 112/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.9042 - mae: 1.7636 - val_loss: 5.3154 - val_mae: 1.8371
Epoch 113/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.9518 - mae: 1.7814 - val_loss: 5.3029 - val_mae: 1.8348
Epoch 114/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.3042 - mae: 1.8287 - val_loss: 5.3001 - val_mae: 1.8339
Epoch 115/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.8676 - mae: 1.7745 - val_loss: 5.3039 - val_mae: 1.8345
Epoch 116/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.1655 - mae: 1.8162 - val_loss: 5.3000 - val_mae: 1.8338
Epoch 117/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/

74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.2190 - mae: 1.9476 - val_loss: 5.7311 - val_mae: 1.8833
Epoch 13/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.6569 - mae: 1.8621 - val_loss: 5.5718 - val_mae: 1.8682
Epoch 14/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.5231 - mae: 1.8730 - val_loss: 5.5180 - val_mae: 1.8648
Epoch 15/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.7565 - mae: 1.9102 - val_loss: 5.4918 - val_mae: 1.8634
Epoch 16/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.6773 - mae: 1.8773 - val_loss: 5.4910 - val_mae: 1.8646
Epoch 17/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.6582 - mae: 1.8974 - val_loss: 5.4818 - val_mae: 1.8639
Epoch 18/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.6406 - mae: 1.9039 - val_loss: 5.4782 - val_mae: 1.8654
Epoch 19/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.4897 - mae: 1.8719 - val_loss: 5.4841 - val_mae: 1.8652
Epoch 20/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.4102 - 

Epoch 66/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.2434 - mae: 1.8267 - val_loss: 5.3611 - val_mae: 1.8480
Epoch 67/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.5177 - mae: 1.8923 - val_loss: 5.3575 - val_mae: 1.8460
Epoch 68/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.1153 - mae: 1.8020 - val_loss: 5.3528 - val_mae: 1.8456
Epoch 69/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.3630 - mae: 1.8339 - val_loss: 5.3584 - val_mae: 1.8466
Epoch 70/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.3280 - mae: 1.8498 - val_loss: 5.3517 - val_mae: 1.8471
Epoch 71/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.4627 - mae: 1.8580 - val_loss: 5.3457 - val_mae: 1.8435
Epoch 72/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.5933 - mae: 1.8753 - val_loss: 5.3479 - val_mae: 1.8457
Epoch 73/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.5763 - mae: 1.8803 - val_loss: 5.3445 - val_mae: 1.8446
Epoch 74/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - lo

...

Epoch 101/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.7468 - mae: 1.9306 - val_loss: 5.4041 - val_mae: 1.8527
Epoch 102/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.8522 - mae: 1.9054 - val_loss: 5.4117 - val_mae: 1.8505
Epoch 103/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.5191 - mae: 1.8535 - val_loss: 5.4044 - val_mae: 1.8512
Epoch 104/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.8516 - mae: 1.9075 - val_loss: 5.4173 - val_mae: 1.8537
Epoch 105/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.6803 - mae: 1.8804 - val_loss: 5.4056 - val_mae: 1.8517
Epoch 106/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.6623 - mae: 1.8952 - val_loss: 5.3993 - val_mae: 1.8530
Epoch 107/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.8056 - mae: 1.9244 - val_loss: 5.4006 - val_mae: 1.8503
Epoch 108/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.6208 - mae: 1.8723 - val_loss: 5.4130 - val_mae: 1.8490
Epoch 109/150
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10.9828 - mae: 2.6152 - val_loss: 5.6526 - val_mae: 1.8802
Epoch 5/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9.1849 - mae: 2.4099 - val_loss: 5.5413 - val_mae: 1.8693
Epoch 6/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.6819 - mae: 2.3613 - val_loss: 5.5173 - val_mae: 1.8696
Epoch 7/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.3743 - mae: 2.2742 - val_loss: 5.4905 - val_mae: 1.8638
Epoch 8/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7.7690 - mae: 2.2002 - val_loss: 5.5001 - val_mae: 1.8648
Epoch 9/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7.7509 - mae: 2.2191 - val_loss: 5.4651 - val_mae: 1.8629
Epoch 10/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7.3946 - mae: 2.1460 - val_loss: 5.4769 - val_mae: 1.8607
Epoch 11/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 7.2815 - mae: 2.1426 - val_loss: 5.4719 - val_mae: 1.8578
Epoch 12/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - l

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.1257 - mae: 1.9553 - val_loss: 5.4680 - val_mae: 1.8555
Epoch 58/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6.0628 - mae: 1.9472 - val_loss: 5.4492 - val_mae: 1.8571
Epoch 59/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.9830 - mae: 1.9507 - val_loss: 5.4433 - val_mae: 1.8680
Epoch 60/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6.1635 - mae: 1.9585 - val_loss: 5.4632 - val_mae: 1.8562
Epoch 61/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.9960 - mae: 1.9603 - val_loss: 5.4368 - val_mae: 1.8621
Epoch 62/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.9750 - mae: 1.9423 - val_loss: 5.4429 - val_mae: 1.8588
Epoch 63/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.9976 - mae: 1.9308 - val_loss: 5.4466 - val_mae: 1.8574
Epoch 64/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.9071 - mae: 1.9188 - val_loss: 5.4566 - val_mae: 1.8555
Epoch 65/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.7459 - mae: 1.9029 - val_loss: 5.4584 - val_mae: 1.8527
Epoch 111/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.8273 - mae: 1.9065 - val_loss: 5.4229 - val_mae: 1.8548
Epoch 112/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.6436 - mae: 1.8808 - val_loss: 5.4167 - val_mae: 1.8574
Epoch 113/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.4358 - mae: 1.8530 - val_loss: 5.4217 - val_mae: 1.8542
Epoch 114/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.7582 - mae: 1.9074 - val_loss: 5.4303 - val_mae: 1.8539
Epoch 115/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.5029 - mae: 1.8586 - val_loss: 5.4193 - val_mae: 1.8563
Epoch 116/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.5484 - mae: 1.8781 - val_loss: 5.4212 - val_mae: 1.8574
Epoch 117/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.9505 - mae: 1.9322 - val_loss: 5.4242 - val_mae: 1.8567
Epoch 118/150
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(learning_rate=0.003, model=<function build_model at 0x000002A021429A80>, n_hidden=1, n_neurons=30),
                   n_iter=20,
                   param_distributions={'model__dropout_rate': [0, 0.1, 0.2,
                                                                0.3, 0.4],
                                        'model__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002A012C2E510>,
                                        'model__n_hidden': [0, 1, 2, 3, 4, 5],
                                        'model__n_neurons': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])},
                   scoring='neg_mean_squared_error')

In [43]:
#print(rnd_search_cv.best_params_)
#print(rnd_search_cv.best_score_)

In [44]:
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)

{'model__dropout_rate': 0.4, 'model__learning_rate': 0.001157866133544249, 'model__n_hidden': 2, 'model__n_neurons': 25}
-5.5286867306164025


In [45]:
best_model = rnd_search_cv.best_estimator_.model

In [46]:
from scikeras.wrappers import KerasRegressor

# Seems like you need to specify some items again when using the scikeras wrapper
keras_reg = KerasRegressor(model=best_model, epochs=n_epoch, batch_size=5, verbose=0, random_state=42,
                           loss='mean_squared_error',optimizer='adam',metrics=['mae'])

# Stop early if model is not getting better after # of patience epochs, restore to best model
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
#checkpoint_cb = keras.callbacks.ModelCheckpoint("Regression_Sequential.keras", save_best_only = True)

keras_reg.fit(X_train_prepared, Y_train, validation_data = (X_valid_prepared, Y_valid),
             callbacks=[early_stopping_cb])


KerasRegressor(
	model=<function build_model at 0x000002A021429A80>
	build_fn=None
	warm_start=False
	random_state=42
	optimizer=adam
	loss=mean_squared_error
	metrics=['mae']
	batch_size=5
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=150
)

In [47]:
from sklearn.model_selection import cross_val_score

# Assuming X and y are your feature matrix and target vector
scores = cross_val_score(keras_reg, X_train_prepared, Y_train, cv=5, scoring='neg_mean_squared_error',verbose=0)

# Convert scores to positive values
mse_scores = -scores
print(f'MSE Scores: {mse_scores}')
print(f'Mean MSE: {np.mean(mse_scores)}')

MSE Scores: [6.02940222 4.9634603  5.30375332 5.54431183 5.85688376]
Mean MSE: 5.539562287606165


In [48]:
X_train_pred = keras_reg.predict(X_train_prepared)
X_test_pred = keras_reg.predict(X_test_prepared)

# Evaluate the model
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(Y_train, X_train_pred)
mae = mean_absolute_error(Y_train, X_train_pred)

print(f'Train MSE: {mse}') # Sensitive to outliers
print(f'Train MAE: {mae}')

mse = mean_squared_error(Y_test, X_test_pred)
mae = mean_absolute_error(Y_test, X_test_pred)

print(f'Test MSE: {mse}') # Sensitive to outliers
print(f'Test MAE: {mae}')

Train MSE: 5.344163711089202
Train MAE: 1.8470409416088096
Test MSE: 5.315119719057888
Test MAE: 1.879280097918077


In [49]:
#df_Predict = pd.read_csv(r"C:\Users\zchodan\OneDrive - Franklin Templeton\Documents\Python\NHL_data\NHL_Data_Over_Under_Predict.csv")
df_Predict = pd.read_csv(r"C:\Users\zanec\OneDrive\Documents\Python\NHL_data\NHL_Data_Over_Under_Predict.csv")

In [50]:
X_real_predict = df_Predict.iloc[:,2:]

np.array((df_Predict.iloc[:,1].values,keras_clf.predict(X_real_predict))).T

NameError: name 'keras_clf' is not defined